# 每小時更新最新新聞

In [1]:
import requests
from lxml import etree
import os
import time
import pymongo
import threading
from tqdm import tqdm

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [2]:
url = "https://www.ettoday.net/news/news-list.htm"
def getDateNews():
    links = []
    tags = []
    response = requests.get(url)
    html = etree.HTML(response.content)
    links = html.xpath("//div[@class='part_list_2']/h3/a/@href")
    links = ["https://www.ettoday.net"+link for link in links]
    tags = html.xpath("//div[@class='part_list_2']/h3/em/text()")
    dates = html.xpath("//div[@class='part_list_2']/h3/span[@class='date']/text()")
    dates = [date.split(" ")[0][5:] for date in dates]
    return tags,links,dates

In [3]:
def getnews(link,D,tag):
    try: 
        response = requests.get(link)
        tag = tag
        html = etree.HTML(response.content,parser=etree.HTMLParser(encoding='utf-8'))
        link = link
        title = html.xpath("//header/h1/text()")
        title = title[0].strip()
        contents = html.xpath("//div[@class='story']/p/text()")
        content = "\n".join([content.strip()for content in contents])
        img_link = html.xpath("//div[@class='story']//img/@src")[0]
        pic = requests.get("https:"+img_link)
        link_id = link.split("/")[-1].split(".")[0]
        time = html.xpath("//time/@datetime")[0]
        time = time.replace("+08:00","").replace("T","-").replace(":","-")
        k_time = re.split("-",time)
        k_time = [int(j) for j in k_time]
        for i in range(len(k_time),9):
            k_time.append(0)
        time = time.mktime(tuple(k_time))
        img_path = "./pic/"+D+"/"+time+"_"+link_id+".png"
        open(img_path,"wb").write(pic.content)
        insert_dict = {"title":title,"content":content,"time":time,"tag":tag,"link":link,"img_path":img_path}
        mycol.update_one({"link":insert_dict["link"]},{"$set":insert_dict},upsert=True)
    except:
        pass

In [4]:
tags,links,dates = getDateNews()
threads = []
m = time.strftime("%m",time.localtime(time.time()))
d = time.strftime("%d",time.localtime(time.time()))
H = time.strftime("%H",time.localtime(time.time()))
date = m+"/"+d+"/"+H+":00"
for i in tqdm(range(len(links)),desc=date):
    tag = tags[i]
    link = links[i]
    if(mycol.find_one({"link":link}) != None):
        continue
    mon,day = dates[i].split("/")
    mon = int(mon)
    day = int(day)
    m = "0"+str(mon) if mon <10 else mon
    d = "0"+str(day) if day <10 else day
    D = "2019"+str(m)+str(d)
    directory = "./pic/"+D
    if not os.path.exists(directory):
        os.makedirs(directory)
#     getnews(link,D,tag)
    threads.append(threading.Thread(target=getnews,args=(link,D,tag)))
    threads[-1].start()
for thread in threads:
    thread.join()
print("finish "+D)

11/11/11:00: 100%|███████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 20.93it/s]


finish 20191111
